In [100]:
import pyspark
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    

In [146]:
leyo_resumen = [        
    ('sumamary_id_1', 'book_id_1', 'user_id_1', '20200102'),
    ('sumamary_id_2', 'book_id_1', 'user_id_2', '20200103'),
    ('sumamary_id_2', 'book_id_1', 'user_id_3', '20200104'),
    ('sumamary_id_3', 'book_id_2', 'user_id_1', '20200202'),
    ('sumamary_id_4', 'book_id_2', 'user_id_4', '20200220'),
    ('sumamary_id_3', 'book_id_2', 'user_id_3', '20200124'),
    ('sumamary_id_4', 'book_id_3', 'user_id_2', '20200210'),
    ('sumamary_id_5', 'book_id_3', 'user_id_4', '20200112'),
    ('sumamary_id_6', 'book_id_4', 'user_id_2', '20200214'),
    ('sumamary_id_7', 'book_id_4', 'user_id_3', '20200214'),
    ('sumamary_id_8', 'book_id_5', 'user_id_2', '20200214'),
    ('sumamary_id_9', 'book_id_9', 'user_id_2', '20200214')
               ]

In [147]:
resumen = [
    ('sumamary_id_1', 'book_id_1', 'user_id_4', 'texto'),
    ('sumamary_id_2', 'book_id_1', 'user_id_5', 'texto'),
    ('sumamary_id_3', 'book_id_2', 'user_id_5', 'texto'),
    ('sumamary_id_4', 'book_id_2', 'user_id_2', 'texto'),
    ('sumamary_id_5', 'book_id_3', 'user_id_2', 'texto'),
    ('sumamary_id_6', 'book_id_4', 'user_id_1', 'texto'),
    ('sumamary_id_7', 'book_id_3', 'user_id_1', 'texto'),
    ('sumamary_id_8', 'book_id_5', 'user_id_1', 'texto'),
   
    
]

In [148]:
r_leyo = sc.parallelize(leyo_resumen);
re = sc.parallelize(resumen);

In [7]:
# obtener los libros mas consultados

In [152]:
leyo_book = r_leyo.map(lambda x: (x[1], 1))\
            .reduceByKey(lambda x,y: x+y)\
            .sortBy(lambda x: -x[1])\
            .zipWithIndex()\
            .filter(lambda x: x[1] < 10)\
            .map(lambda x: (x[0][0], x[1]+1))\

# cuento libros consultados
#oredeno de mayor a menor
#agrego index
#quito los que no son top 10
#como el index va de 0-9 sumo 1

sumary_book = r_leyo.map(lambda x: ((x[1],x[0]), 1))\
                    .reduceByKey(lambda x,y: x+y)\
                    .map(lambda x: (x[0][0], (x[0][1], x[1])))\
                    .groupByKey()\
                    .map(lambda x: (x[0], sorted(x[1], key=lambda tup: -tup[1])))\

#mapeo, libro-sumary
#cuento lecturas de resumen por libro
#mapeo sumary con la cantidad de veces que se leyo
#hago lista
#ondeno lista por cantidad de lecturas

book_sumary = leyo_book.join(sumary_book)\
                        .map(lambda x: (x[0], x[1][0], x[1][1][0][0]))\

#joineo rdd
#tomo el libro-index-summary_mas_leido
#  x[1][1][1][0], #  x[1][1][2][0]

book_sumary.collect()

[('book_id_1', 1, 'sumamary_id_2'),
 ('book_id_2', 2, 'sumamary_id_3'),
 ('book_id_4', 3, 'sumamary_id_7'),
 ('book_id_5', 5, 'sumamary_id_8'),
 ('book_id_9', 6, 'sumamary_id_9'),
 ('book_id_3', 4, 'sumamary_id_4')]

In [86]:
#a = book_sumary.takeOrdered(10, key= lambda x: -x[1][0])

In [108]:
#list_final = []
#i = 1
#for x in a:
#    elem = [x[0], i]
#    for e in x[1][1]:
#        elem.append(e[0])
#    i+=1
#    list_final.append(elem[:5])

In [43]:
import pandas as pd

In [95]:
data = pd.DataFrame(list_final, columns=["book_id", "top_10_position", "sumary_id_1", "summary_id_2", "summary_id_3"])

In [96]:
data

,book_id,top_10_position,sumary_id_1,summary_id_2,summary_id_3
0,book_id_1,1,sumamary_id_2,sumamary_id_1,s2
1,book_id_2,2,sumamary_id_3,sumamary_id_4,s2
2,book_id_4,3,sumamary_id_7,sumamary_id_6,s2
3,book_id_3,4,sumamary_id_4,sumamary_id_5,s2
4,book_id_5,5,sumamary_id_8,s2,s3


In [53]:
#data.to_csv("data.csv")